In [1]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import sys
import numpy as np
import getpass
import matplotlib.pyplot as plt

In [2]:
#add the github path that will run the notebooks: 
username = getpass.getuser() # gets the username from the current comp 
print (username)

gilmandelbaum


In [3]:
sys.path.insert(0,'/Users/{:}/github/analysis-pipeline-for-photometry_ex/Nb_data_set/3.data_set_correlations'.format(username))

In [4]:
sys.path[0]

'/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/3.data_set_correlations'

In [5]:
#import data_set_correlations_helper_functions as dschf

# Change based on analysis: 

In [6]:
seq_str = '0a1b2a3b4abcd5a6a7a'
period_of_interest = "combined_periods_1"

In [7]:
HowManyBack=1
folderName="left_dual_color_photo_full_data_set_after_inspection" #folder where the plot data_set will be saved 
path = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis_3/"

# Folder name of analysis: 

In [8]:
analysis_type ="data_set_correlations"

In [9]:
sys.executable

'/anaconda3/bin/python3'

# Import the data and where to save it 

In [10]:
path_to_save = path+"/"+folderName+"/"+seq_str #also for loading

In [11]:
path_to_save

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis_3//left_dual_color_photo_full_data_set_after_inspection/0a1b2a3b4abcd5a6a7a'

In [12]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(path_to_save,analysis_type,period_of_interest)

In [13]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(path_to_save,analysis_type,period_of_interest)
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)

# Import data_trial

In [14]:
%%time
root = Path(path_to_save)
d = "data_object_data_trial"+"_"+seq_str+'.pickle'
my_path = root / d 
fileToOpen = open(my_path, 'rb')
data_trial_sessions = pickle.load(fileToOpen)

CPU times: user 50.2 ms, sys: 11.5 ms, total: 61.7 ms
Wall time: 176 ms


# Extract selection time 

In [15]:
def extract_behaviorFeature_perSession(data_trial_sessions,behavior_measure):
    behavior_measure_list=[]
    for data_trial in data_trial_sessions:
        behavior_measure_list.append(pd.DataFrame(data_trial[behavior_measure]))
    return behavior_measure_list

In [16]:
data_trial_sessions_rt=extract_behaviorFeature_perSession(data_trial_sessions,'tSelection')

# Import data_photometry

In [17]:
%%time
root = Path(path_to_save)
d = "data_object_photometry"+"_"+seq_str+'.pickle'
my_path = root / d 
fileToOpen = open(my_path, 'rb')
data_set = pickle.load(fileToOpen)

CPU times: user 13.9 s, sys: 1.21 s, total: 15.1 s
Wall time: 16 s


# generate data sets of interest

In [18]:
def extract_data_of_interest_ipsi_contra_per_session_helper (PhotoData_perTrial_channels,rl,tt,period,cell_type):
    ipsi_next = PhotoData_perTrial_channels[rl][0][tt][period][cell_type]
    contra_next = PhotoData_perTrial_channels[rl][1][tt][period][cell_type]
    return (ipsi_next,contra_next)
    


def extract_data_of_interest_ipsi_contra_data_set (PhotoData_perTrial_channels_data_set,
                                                   rl,tt,period,cell_type): 
    
    ipsi_contra_data_set = []
    
    for PhotoData_perTrial_channels in PhotoData_perTrial_channels_data_set:
        ipsi_contra = extract_data_of_interest_ipsi_contra_per_session_helper (PhotoData_perTrial_channels,rl,tt,period,cell_type)
        ipsi_contra_data_set.append(ipsi_contra)
        
    return (ipsi_contra_data_set)

In [19]:
l_ls_period_d1_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           1,0,
                                                                           period_of_interest,"d1 L")

In [20]:
l_ls_period_d2_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           1,0,
                                                                           period_of_interest,"d2 L")

In [21]:
l_wr_period_d1_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           1,3,
                                                                           period_of_interest,"d1 L")

In [22]:
l_wr_period_d2_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           1,3,
                                                                           period_of_interest,"d2 L")

In [23]:
def extract_signal_per_session (data_state_sessions,function):
    maxSig_sessions=[]
    
    for session in data_state_sessions:
        max_ipsi_next = function(session[0])
        max_contra_next = function(session[1])
        
        maxSig_sessions.append((max_ipsi_next,max_contra_next))

    return maxSig_sessions

In [24]:
def extract_mean_max_signal(session_ipsi_or_contra):
    list_max_value=[]
    df = pd.DataFrame()
    
    idx_max_signal_list = list(session_ipsi_or_contra.loc[200:500,:].idxmax())
    
    for c,i in enumerate (idx_max_signal_list): #c is the count, i is the index of max value in each session avg 
        max_signal_per_trial = session_ipsi_or_contra.iloc[(i-10):(i+11),c].mean()
        list_max_value.append(max_signal_per_trial)
        
    df['max_signal'] = list_max_value
    
    trial_number = list(session_ipsi_or_contra.keys())
    df.index = trial_number
    
    return df

In [25]:
l_ls_period_d1_maxSig=extract_signal_per_session(l_ls_period_d1_data_set,extract_mean_max_signal)

In [26]:
l_ls_period_d2_maxSig=extract_signal_per_session(l_ls_period_d2_data_set,extract_mean_max_signal)

In [27]:
l_wr_period_d1_maxSig=extract_signal_per_session(l_wr_period_d1_data_set,extract_mean_max_signal)

In [28]:
l_wr_period_d2_maxSig=extract_signal_per_session(l_wr_period_d2_data_set,extract_mean_max_signal)

In [29]:
def combine_resTime_maxSignal(resTime,maxSignal):
    
    trial_num=list(maxSignal.index) #get the index of the max signal (this is the trial numbers that we
    #are intersted in). 
    
    resTime_list = resTime.loc[trial_num] #get the response time of interest as a data frame 
    
    #combine the above:
    df_resTime_maxSig = pd.concat([resTime_list,maxSignal],axis=1)

    df_resTime_maxSig = df_resTime_maxSig
    
    return df_resTime_maxSig

In [30]:
def get_maxSig_resTime(processed_photo_sessions, processed_beh_sessions,column_header):
    df_time_max_signal_ipsi= pd.DataFrame()
    df_time_max_signal_contra = pd.DataFrame()   
    
    for photo,beh in zip(processed_photo_sessions,processed_beh_sessions):
        
        
        df_ipsi = combine_resTime_maxSignal(beh, photo[0])
        df_contra = combine_resTime_maxSignal(beh, photo[1])

        df_time_max_signal_ipsi = pd.concat([df_time_max_signal_ipsi, df_ipsi])
        df_time_max_signal_contra = pd.concat([df_time_max_signal_contra, df_contra])       
    
    
    
    df_time_max_signal_ipsi = df_time_max_signal_ipsi.reset_index().drop(["index"],axis=1)
    df_time_max_signal_contra = df_time_max_signal_contra.reset_index().drop(["index"],axis=1)
    
    
    #make second header: 
    df_time_max_signal_ipsi.columns = [[(column_header+"_ipsi"),(column_header+"_ipsi")], df_time_max_signal_ipsi.columns]
    df_time_max_signal_contra.columns = [[(column_header+"_contra"),(column_header+"_contra")], df_time_max_signal_contra.columns]


    
    
    return(df_time_max_signal_ipsi,df_time_max_signal_contra)        

In [31]:
l_ls_period_d1_resTime_maxSig=get_maxSig_resTime(l_ls_period_d1_maxSig, data_trial_sessions_rt,"l_ls_d1")

In [32]:
l_ls_period_d2_resTime_maxSig=get_maxSig_resTime(l_ls_period_d2_maxSig,data_trial_sessions_rt,"l_ls_d2")

In [33]:
l_wr_period_d1_resTime_maxSig=get_maxSig_resTime(l_wr_period_d1_maxSig,data_trial_sessions_rt,"l_wr_d1")

In [34]:
l_wr_period_d2_resTime_maxSig=get_maxSig_resTime(l_wr_period_d2_maxSig,data_trial_sessions_rt,"l_wr_d2")

In [35]:
result = pd.concat([l_ls_period_d1_resTime_maxSig[0],l_ls_period_d1_resTime_maxSig[1],
                    l_ls_period_d2_resTime_maxSig[0],l_ls_period_d2_resTime_maxSig[1],
                    l_wr_period_d1_resTime_maxSig[0],l_wr_period_d1_resTime_maxSig[1],
                    l_wr_period_d2_resTime_maxSig[0],l_wr_period_d2_resTime_maxSig[1]],axis=1)

In [39]:
result.head(5)

l_ls_d1_ipsi            l_ls_d1_contra            l_ls_d2_ipsi             \
    tSelection max_signal     tSelection max_signal   tSelection max_signal   
0        415.0   1.754610           85.0   1.840126        415.0   1.382829   
1        235.0   1.000941           90.0   2.305714        235.0   1.702935   
2         65.0   1.075767           50.0   2.067732         65.0   0.600126   
3        200.0   2.238119           50.0   2.466278        200.0   2.502666   
4         75.0   0.421078          175.0   2.451018         75.0   1.344817   

  l_ls_d2_contra            l_wr_d1_ipsi            l_wr_d1_contra             \
      tSelection max_signal   tSelection max_signal     tSelection max_signal   
0           85.0   0.783740         80.0   2.557436            180   1.879736   
1           90.0   4.302031        215.0   2.048357             70   0.473158   
2           50.0   1.281789         55.0   1.793179            185   0.504684   
3           50.0   2.217883        210.0   0.879416            230   1.299703   
4          175.0   1.308882         85.0   1.748262            185   1.280136   

  l_wr_d2_ipsi            l_wr_d2_contra             
    tSelection max_signal     tSelection max_signal  
0         80.0   2.868773            180   1.922100  
1        215.0   1.005391             70   1.482910  
2         55.0   2.254109            185   0.908911  
3        210.0   1.441658            230   0.936278  
4         85.0   1.567134            185   2.008310

In [42]:
result.to_csv(cache_dir_plt+"/data_set_correlations_rt_vs_signal_magnitude_left_side.csv")

In [41]:
cache_dir_plt+"/data_set_correlations_rt_vs_signal_magnitude_left_side.csv"

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis_3//left_dual_color_photo_full_data_set_after_inspection/0a1b2a3b4abcd5a6a7a/data_set_correlations/combined_periods_1/data_set_correlations_rt_vs_signal_magnitude_left_side.csv'